## try img rag

* imgファイルをもとにしたRAG関連の検証

In [1]:
!pip install python-dotenv
!pip install pandas
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = ".env"
load_dotenv(dotenv_path)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

from IPython.display import display, Markdown

### 定数

In [3]:
QUERY1 = "2023年で即席めんの一人当たりの年間消費量が最も多い国はどこか。"

In [4]:
IMG1 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/日清食品ホールディングス株式会社_69_70.png"

In [5]:
QUERY2 = "東洋エンジニアリングの取締役が有する専門的知見や経験分野として多く選択されている項目は、『法務・法規則』と『人事・労務』のどちらでしょうか？"

In [6]:
IMG2 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/東洋エンジニアリング株式会社_40_49.png"

In [7]:
QUERY3 = "全国保証株式会社の有給休暇取得日数は2023/3期対比2024/3期は何日増加しましたか？"

In [8]:
IMG3_1 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/全国保証株式会社_16_42.png"

In [9]:
IMG3_2 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/全国保証株式会社_17_42.png"

In [10]:
IMG3_3 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/全国保証株式会社_11_42.png"

In [11]:
IMG3_4 = "https://signate-fdua-rag.s3.ap-northeast-1.amazonaws.com/全国保証株式会社_12_42.png"

### IMGファイルから直接回答を生成する

* 答えが記載されているページのみを入力し、質問に回答できるか検証
* 実際にはどのページに答えが記載されているか不明なので、質問に関する企業レポートを全ページ or 数ページずつ入力する必要がある、さらにその結果から結論を出す必要もある

In [12]:
class Answer(BaseModel):
    answer: str = Field(description="画像をもとに考えた回答")
    reason: str = Field("回答の理由")

In [13]:
def answer_from_img(query, img_url):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                [
                    {
                        "type": "text",
                        "text": (
                            "ユーザが入力した画像をもとに、質問に回答してください。\n"
                            "回答と合わせて理由も教えてください。\n"
                            "ステップバイステップに考えましょう。\n\n"
                            "質問:\n"
                            "{query}"
                        ),
                    },
                    {
                        "type": "image_url",
                        "image_url": "{img_url}",
                    },
                ]
            )
        ]
    )

    model = ChatOpenAI(model="gpt-4o")

    chain = prompt | model.with_structured_output(Answer)

    answer = chain.invoke({
        "query": query,
        "img_url": img_url,
    })

    return answer

In [14]:
answer_from_img(QUERY1, IMG1)

Answer(answer='韓国が2023年で即席めんの一人当たりの年間消費量が最も多い国です。', reason='画像のデータによると、韓国の2023年の一人当たりの即席めん消費量は78.3食であり、他の国と比較して最も高い値です。')

* 正解はベトナムの80.9食なので誤り
* ただし、韓国の2023年の一人当たりの即席めん年間消費量は78.3食で2位なので、表は一定解釈が出来てそう

In [15]:
answer_from_img(QUERY2, IMG2)

Answer(answer='『法務・法規則』の方が多く選択されています。', reason='画像のスキルマトリックスを見ると、『法務・法規則』の項目には複数の取締役が該当している一方、『人事・労務』の項目にはそれよりも少ない取締役しか該当していません。このことから、『法務・法規則』の方が多くの取締役によって選択されていることがわかります。')

* 法務・法規則が3人、人事・労務が2人なので正解

In [16]:
answer_from_img(QUERY3, IMG3_1)

Answer(answer='全国保証株式会社の有給休暇取得日数は、2023/3期で11.8日、2024/3期で12.9日です。したがって、2024/3期は2023/3期に比べて1.1日増加しました。', reason='画像の中で、有給休暇取得日数のグラフを確認しました。2023/3期では11.8日、2024/3期では12.9日と示されています。したがって、2024/3期は2023/3期に比べて1.1日増加しています。')

* 2023/3期では11.8日、2024/3期では12.9日、差分は1.1日なので正解

### 複数のIMGファイルから直接回答を生成する

* 複数画像からの回答はできそう
* P.1-3、P.2-4、...とウィンドウをずらしていけばよさそう

In [17]:
def answer_from_multiple_imgs(query, img_url_1, img_url_2):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                [
                    {
                        "type": "text",
                        "text": (
                            "ユーザが入力した2つの画像をもとに、質問に回答してください。\n"
                            "回答と合わせて理由も教えてください。\n"
                            "回答に自信がない場合、無理に答えないでください。\n"
                            "ステップバイステップに考えましょう。\n\n"
                            "質問:\n"
                            "{query}"
                        ),
                    },
                    {
                        "type": "image_url",
                        "image_url": "{img_url_1}",
                    },
                    {
                        "type": "image_url",
                        "image_url": "{img_url_2}",
                    },
                ]
            )
        ]
    )

    model = ChatOpenAI(model="gpt-4o")

    chain = prompt | model.with_structured_output(Answer)

    answer = chain.invoke({
        "query": query,
        "img_url_1": img_url_1,
        "img_url_2": img_url_2,
    })

    return answer

In [18]:
answer_from_multiple_imgs(QUERY3, IMG3_1, IMG3_2)

Answer(answer='2023/3期の有給休暇取得日数は11.8日、2024/3期は12.9日です。これにより、2024/3期は2023/3期よりも1.1日増加しています。', reason='画像のデータから、有給休暇取得日数が2023/3期では11.8日、2024/3期では12.9日であることが確認できます。この差を計算すると、1.1日増加しています。')

### 複数の回答から結論の回答を生成する

* 妥当そうな回答が1つであれば結論を求められそう

In [19]:
class Conclusion(BaseModel):
    conclusion: str = Field(description="複数の回答をもとに、最終的に結論とした導いた端的な回答")
    reason: str = Field("結論とした理由")

In [20]:
def conclude(query, ans1, ans2):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                [
                    {
                        "type": "text",
                        "text": (
                            "質問に対してAIがいくつかの画像を参照して生成した回答があります。\n"
                            "質問と複数の回答を踏まえ、結論として最終的な回答だけを単語で端的に教えてください。\n\n"
                            "質問:\n"
                            "{query}\n\n"
                            "回答1:\n"
                            "{ans1}\n\n"
                            "回答2:\n"
                            "{ans2}"
                        )
                    }
                ]
            )
        ]
    )

    model = ChatOpenAI(model="gpt-4o-mini")

    chain = prompt | model.with_structured_output(Conclusion)

    conclusion = chain.invoke({
        "query": query,
        "ans1": ans1,
        "ans2": ans2,
    })

    return conclusion

In [21]:
ans1 = answer_from_multiple_imgs(QUERY3, IMG3_1, IMG3_2)
ans1

Answer(answer='2023/3期の有給休暇取得日数は11.8日で、2024/3期は12.9日です。したがって、有給休暇取得日数は1.1日増加しました。', reason='画像のデータを確認すると、2023/3期の有給休暇取得日数は11.8日と記載されています。2024/3期の有給休暇取得日数は12.9日です。このため、2024/3期は2023/3期と比較して1.1日増加しています。')

In [22]:
ans2 = answer_from_multiple_imgs(QUERY3, IMG3_3, IMG3_4)
ans2

Answer(answer='提供された画像には、全国保証株式会社の有給休暇取得日数に関する情報は含まれていません。そのため、増加日数を判断することはできません。', reason='回答の理由')

In [23]:
conclude(QUERY3, ans1.answer, ans2.answer)

Conclusion(conclusion='1.1日', reason='回答1からの情報')

### IMGファイルから文字起こし

* 精度はあまりよくないが、表やグラフであることは解釈できてそう
* AI Document Intelligenceと合わせ技で精度よくなるかも[（参考）](https://dev.classmethod.jp/articles/fix-claude3-text-recognition-mistake-with-azure-document-intelligence/)

In [24]:
class Transcription(BaseModel):
    markdown_text: str = Field(description="画像をmarkdown形式で文字起こしした結果")

In [25]:
def transcribe(img_url):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "user",
                [
                    {
                        "type": "text",
                        "text": (
                            "ユーザが入力した画像について、markdown形式で文字起こししてください。\n"
                            "表についてもmarkdown形式で文字起こししてください。\n"
                            "グラフや画像については、言葉で簡潔に説明してください。\n"
                            "内容は漏れなく文字起こししてください。\n"
                        )
                    },
                    {
                        "type": "image_url",
                        "image_url": "{img_url}"
                    },
                ]
            )
        ]
    )

    model = ChatOpenAI(model="gpt-4o")

    chain = prompt | model.with_structured_output(Transcription)

    transcription = chain.invoke({
        "img_url": img_url
    })

    return transcription

In [26]:
display(Markdown(transcribe(IMG1).markdown_text))

## データ

### 即席めんの世界市場データ

| 国名／地域   | 人口*1 (100万人) | GDP*2 (10億米ドル) | 一人当たりGDP*2 (米ドル) | 総需要*3, *4 (億食) |
|--------------|----------------|-----------------|---------------------|-----------------|
|              |                |                 |                     | CY2014 | CY2015 | CY2016 | CY2017 | CY2018 | CY2019 | CY2020 | CY2021 | CY2022 | CY2023 | 一人当たりの消費量*4 (食) |
| 中国         | 1,411.4        | 17,701          | 12,541              | 444.0  | 404.3  | 385.2  | 389.6  | 402.5  | 414.5  | 463.6  | 439.9  | 450.7  | 422.1  | 29.7  |
| 香港         | 7.5            | 386             | 51,168              | 44.0   | 40.4   | 38.5   | 38.6   | 40.2   | 41.5   | 46.3   | 43.9   | 45.7   | 42.1   | 29.7  |
| インドネシア | 277.4          | 1,417           | 5,109               | 134.3  | 132.0  | 130.1  | 126.2  | 125.4  | 125.2  | 126.4  | 132.7  | 142.6  | 145.4  | 52.4  |
| ベトナム     | 100.4          | 433             | 4,316               | 50.0   | 48.0   | 49.2   | 50.6   | 52.0   | 54.4   | 70.3   | 85.6   | 84.8   | 81.3   | 80.9  |
| インド       | 1,428.6        | 3,732           | 2,612               | 53.4   | 32.6   | 42.7   | 54.2   | 60.6   | 67.3   | 67.3   | 75.6   | 75.8   | 66.8   | 6.1   |
| 日本         | 124.6          | 4,231           | 33,950              | 55.0   | 55.4   | 56.6   | 56.6   | 57.8   | 56.3   | 59.7   | 58.5   | 59.8   | 58.4   | 46.9  |
| フィリピン   | 112.9          | 436             | 3,859               | 33.2   | 34.8   | 34.0   | 37.5   | 39.8   | 38.5   | 44.7   | 44.4   | 42.9   | 43.9   | 38.9  |
| 韓国         | 51.6           | 1,709           | 33,147              | 35.9   | 36.5   | 38.8   | 37.4   | 38.2   | 39.0   | 41.3   | 37.9   | 39.5   | 40.4   | 78.3  |
| タイ         | 70.2           | 512             | 7,298               | 30.7   | 30.7   | 33.6   | 33.9   | 34.6   | 35.7   | 37.1   | 36.3   | 38.7   | 39.5   | 56.3  |
| マレーシア   | 33.1           | 431             | 13,034              | 13.4   | 13.7   | 13.9   | 13.1   | 13.7   | 14.5   | 15.7   | 15.8   | 15.5   | 16.4   | 49.6  |
| カンボジア   | 16.2           | 31              | 1,916               | 2.5    | 2.7    | 2.8    | 3.3    | 3.3    | 3.5    | 3.7    | 4.1    | 4.0    | 4.2    | 26.1  |
| シンガポール | 5.7            | 497             | 87,884              | 1.3    | 1.3    | 1.3    | 1.3    | 1.3    | 1.3    | 1.6    | 1.4    | 1.3    | 1.3    | 23.1  |
| アジア計     | -              | -               | -                   | 887.9  | 827.8  | 828.2  | 847.9  | 876.0  | 900.3  | 985.5  | 991.1  | 1,016.6| 995.5  | -     |
| オセアニア計 | -              | -               | -                   | 4.4    | 4.5    | 4.6    | 4.8    | 5.0    | 5.1    | 5.3    | 5.5    | 5.5    | 6.2    | -     |
| アメリカ     | 335.1          | 26,950          | 80,412              | 42.8   | 40.8   | 41.2   | 41.3   | 45.2   | 46.3   | 50.5   | 49.8   | 51.5   | 51.0   | 15.2  |
| 北米計       | -              | -               | -                   | 44.7   | 42.7   | 43.1   | 43.2   | 47.1   | 48.2   | 52.4   | 51.8   | 53.5   | 53.0   | -     |
| ブラジル     | 204.2          | 2,127           | 10,413              | 23.7   | 23.7   | 23.7   | 22.5   | 23.9   | 24.2   | 27.2   | 28.5   | 28.3   | 25.5   | 12.5  |
| メキシコ     | 131.2          | 1,811           | 13,804              | 9.0    | 8.5    | 8.9    | 9.1    | 11.2   | 11.7   | 11.6   | 13.6   | 15.1   | 15.5   | 11.8  |
| 中南米計     | -              | -               | -                   | 36.7   | 36.0   | 37.1   | 36.9   | 39.6   | 40.7   | 43.6   | 47.5   | 49.3   | 47.5   | -     |
| ロシア       | 143.2          | 1,862           | 13,006              | 19.4   | 18.4   | 15.7   | 17.8   | 18.5   | 19.1   | 20.0   | 21.0   | 22.0   | 22.0   | 15.4  |
| イギリス     | 68.1           | 3,332           | 48,913              | 3.6    | 3.6    | 3.0    | 3.0    | 3.2    | 3.5    | 3.8    | 4.2    | 4.2    | 4.0    | 6.0   |
| ドイツ       | 83.9           | 4,430           | 52,824              | 1.8    | 1.9    | 2.3    | 2.3    | 3.2    | 3.3    | 3.7    | 3.9    | 3.9    | 3.9    | 4.6   |
| フランス     | 65.8           | 3,049           | 46,315              | 0.6    | 0.6    | 0.6    | 0.9    | 0.8    | 0.9    | 1.0    | 1.1    | 1.1    | 1.1    | 1.7   |
| ハンガリー   | 9.7            | 204             | 21,076              | 0.4    | 0.4    | 0.3    | 0.4    | 0.3    | 0.4    | 0.4    | 0.4    | 0.4    | 0.4    | 4.0   |
| ヨーロッパ計 | -              | -               | -                   | 36.7   | 34.6   | 33.6   | 32.2   | 35.6   | 37.0   | 39.1   | 41.7   | 39.7   | 40.2   | -     |
| アフリカ計   | -              | -               | -                   | 19.8   | 20.1   | 21.4   | 23.0   | 24.3   | 26.2   | 33.7   | 36.7   | 40.1   | 49.5   | -     |
| 合計         | -              | -               | -                   | 1,093.6| 974.9  | 975.2  | 1,001.1| 1,036.2| 1,064.2| 1,165.6| 1,181.8| 1,212.0| 1,202.1| -     |

### [出所]
*1 International Monetary Fund(IMF)  *2 世界ラーメン協会(WINA)  *3 一部、 漸次修正している国・地域があります  *4 人口と2023年の総需要量に基づき算出

---
ページ番号: 69


In [27]:
display(Markdown(transcribe(IMG3_1).markdown_text))

### 非財務ハイライト

#### 従業員数／男女別
- **292人**／男性**204人**・女性**88人**
  - グラフ：2020/3から2024/3にかけての男性と女性の人数の推移を示す棒グラフ。

#### 新規採用者数／男女別
- **25人**／男性**16人**・女性**9人**
  - グラフ：2020/3から2024/3にかけての新規採用者数の推移を示す棒グラフ。

#### 男女別平均勤続年数
- 男性**11.7年**・女性**8.7年**
  - グラフ：2020/3から2024/3にかけての勤続年数の推移を示す折れ線グラフ。

#### 男女別育休取得率
- 男性**100%**・女性**100%**
  - グラフ：2020/3から2024/3にかけての育休取得率の推移を示す折れ線グラフ。

#### 障がい者雇用率
- **2.59%**
  - グラフ：2020/3から2024/3にかけての雇用率の推移を示す折れ線グラフ。

#### 有給休暇取得日数
- **12.9日**
  - グラフ：2020/3から2024/3にかけての有給休暇取得日数の推移を示す棒グラフ。

#### 育児休業復職率
- **100%**
  - グラフ：2020/3から2024/3にかけての復職率の推移を示す折れ線グラフ。

#### 社内取締役・社外取締役人数／社外取締役比率
- 社外取締役比率**42.9%**
  - グラフ：2020/3から2024/3にかけての取締役人数と社外取締役比率の推移を示す棒グラフ。

#### 有給休暇取得率
- **66.8%**
  - グラフ：2020/3から2024/3にかけての取得率の推移を示す折れ線グラフ。

#### 1ヶ月当たり法定外労働時間
- **16.2時間**
  - グラフ：2020/3から2024/3にかけての労働時間の推移を示す折れ線グラフ。

#### 社内監査役・社外監査役人数／社外監査役比率
- 社外監査役比率**75.0%**
  - グラフ：2020/3から2024/3にかけての監査役人数と社外監査役比率の推移を示す棒グラフ。

#### 女性役員比率
- **18.2%**
  - グラフ：2020/3から2024/3にかけての役員比率の推移を示す折れ線グラフ。

---
- **Page 29** | **ZENKOKU HOSHO Co., Ltd.**
- **Page 30** | **Integrated Report 2024**